# router testor

In [46]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [47]:
from langchain_core.pydantic_v1 import BaseModel, Field

class ToPresentProductList(BaseModel):
    """Transfers work to a specialized assistant to present product list"""

    task_to_request: str = Field(description="상품 목록을 제시하는 전문가에게 요청하는 작업")
    evaluation_for_choice: str = Field(description="ToPresentProductList 도구 사용에 대한 평가. 적절하면 good, 적절하지 않으면 bad로 평가.")
    rationale_for_evaluation: str = Field(description="도구 사용 평가에 대한 근거")

In [48]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

routing_criteria = """
    이전 대화에서 판매 중인 상품 목록을 제시했는지 확인해.
    만약 판매중인 상품 목록을 제시하지 않았다면 ToPresentProductList를 사용해
    고객이 주문할 품목을 말했다면 TodOrderRequestConfirmation를 사용해.
    고객이 주문을 진행하려는 내용에 동의했다면 create_order를 사용해.
    """

order_create_router_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 주문을 생성하는 유능한 주문봇이야.
            적절한 도구를 사용해 고객의 요청을 처리해.
            고객이 응답이 필요할 때는 도구를 사용하지 말고 고객에게 응답을 부탁해.

            {routing_criteria}

            user_id: {user_info},
            input_message: {input_message}
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
).partial(routing_criteria=routing_criteria)

tools = [ToPresentProductList]
order_create_router_runnable = order_create_router_prompt | llm.bind_tools(tools)

In [49]:

input_message = "주문할게요"
user_info = "1"
messages = []

output = order_create_router_runnable.invoke({"input_message": input_message,
                                              "user_info": user_info,
                                              "messages": messages,
                                              }),

output

(AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ggiGjJ0OszaOnkk8a1dMgm1w', 'function': {'arguments': '{"task_to_request":"제품 목록을 제시해주세요.","evaluation_for_choice":"good","rationale_for_evaluation":"주문을 진행하기 전에 고객에게 제품 목록을 보여주어 선택할 수 있도록 도와야 합니다."}', 'name': 'ToPresentProductList'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 375, 'total_tokens': 457}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e8d26f40-4b24-47e4-82e0-7c6524bc3524-0', tool_calls=[{'name': 'ToPresentProductList', 'args': {'task_to_request': '제품 목록을 제시해주세요.', 'evaluation_for_choice': 'good', 'rationale_for_evaluation': '주문을 진행하기 전에 고객에게 제품 목록을 보여주어 선택할 수 있도록 도와야 합니다.'}, 'id': 'call_ggiGjJ0OszaOnkk8a1dMgm1w'}]),)

In [50]:
output[0].tool_calls

[{'name': 'ToPresentProductList',
  'args': {'task_to_request': '제품 목록을 제시해주세요.',
   'evaluation_for_choice': 'good',
   'rationale_for_evaluation': '주문을 진행하기 전에 고객에게 제품 목록을 보여주어 선택할 수 있도록 도와야 합니다.'},
  'id': 'call_ggiGjJ0OszaOnkk8a1dMgm1w'}]

In [51]:

input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = []

output = order_create_router_runnable.invoke({"input_message": input_message,
                                              "user_info": user_info,
                                              "messages": messages,
                                              }),

output

(AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aiXVnw900yvys0H60EX3oxHn', 'function': {'arguments': '{"task_to_request":"제품 목록을 제시해주세요.","evaluation_for_choice":"good","rationale_for_evaluation":"상품 목록을 제공하여 고객이 주문을 진행할 수 있도록 도와줄 수 있습니다."}', 'name': 'ToPresentProductList'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 386, 'total_tokens': 464}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fb065fdf-3b08-44c2-bbe0-654d27f3384e-0', tool_calls=[{'name': 'ToPresentProductList', 'args': {'task_to_request': '제품 목록을 제시해주세요.', 'evaluation_for_choice': 'good', 'rationale_for_evaluation': '상품 목록을 제공하여 고객이 주문을 진행할 수 있도록 도와줄 수 있습니다.'}, 'id': 'call_aiXVnw900yvys0H60EX3oxHn'}]),)

In [52]:
output[0].tool_calls

[{'name': 'ToPresentProductList',
  'args': {'task_to_request': '제품 목록을 제시해주세요.',
   'evaluation_for_choice': 'good',
   'rationale_for_evaluation': '상품 목록을 제공하여 고객이 주문을 진행할 수 있도록 도와줄 수 있습니다.'},
  'id': 'call_aiXVnw900yvys0H60EX3oxHn'}]

In [53]:
product_list = [
    {
        "product_name": "떡케익5호",
        "quantity": 1,
        "price": 54000
    },
    {
        "product_name": "무지개 백설기 케익",
        "quantity": 1,
        "price": 51500
    },
    {
        "product_name": "미니 백설기",
        "quantity": 35,
        "price": 31500
    },
    {
        "product_name": "개별 모듬팩",
        "quantity": 1,
        "price": 13500
    }
]

In [54]:
from langchain_core.messages import HumanMessage, AIMessage

input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    HumanMessage(content="주문할게요"), 
    AIMessage(content=product_list)
    ]

try:
    output = order_create_router_runnable.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

Exception: Error code: 400 - {'error': {'message': "Missing required parameter: 'messages[2].content[0].type'.", 'type': 'invalid_request_error', 'param': 'messages[2].content[0].type', 'code': 'missing_required_parameter'}}


In [55]:
from langchain_core.messages import HumanMessage, AIMessage

input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    HumanMessage(content="주문할게요"), 
    AIMessage(content=product_list)
    ]

try:
    output = order_create_router_runnable.invoke({
        "input_message": HumanMessage(content=input_message),
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

Exception: Error code: 400 - {'error': {'message': "Missing required parameter: 'messages[2].content[0].type'.", 'type': 'invalid_request_error', 'param': 'messages[2].content[0].type', 'code': 'missing_required_parameter'}}


In [56]:
input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    HumanMessage(content="주문할게요"), 
    AIMessage(content=product_list)
    ]


compiled_prompt = order_create_router_prompt.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
print(compiled_prompt.messages)

[SystemMessage(content='\n            너는 주문을 생성하는 유능한 주문봇이야.\n            적절한 도구를 사용해 고객의 요청을 처리해.\n            고객이 응답이 필요할 때는 도구를 사용하지 말고 고객에게 응답을 부탁해.\n\n            \n    이전 대화에서 판매 중인 상품 목록을 제시했는지 확인해.\n    만약 판매중인 상품 목록을 제시하지 않았다면 ToPresentProductList를 사용해\n    고객이 주문할 품목을 말했다면 TodOrderRequestConfirmation를 사용해.\n    고객이 주문을 진행하려는 내용에 동의했다면 create_order를 사용해.\n    \n\n            user_id: 1,\n            input_message: 떡케익 2개 주문할게요\n            '), HumanMessage(content='주문할게요'), AIMessage(content=[{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000}, {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500}, {'product_name': '미니 백설기', 'quantity': 35, 'price': 31500}, {'product_name': '개별 모듬팩', 'quantity': 1, 'price': 13500}])]


In [57]:
input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    "주문할게요", 
    product_list
    ]


try:
    output = order_create_router_runnable.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

Exception: Unsupported message type: <class 'list'>


In [58]:
from langchain_core.messages import BaseMessage


input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    "주문할게요", 
    product_list
    ]
message = BaseMessage(messages)


try:
    output = order_create_router_runnable.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

ValidationError: 4 validation errors for BaseMessage
content
  str type expected (type=type_error.str)
content -> 1
  str type expected (type=type_error.str)
content -> 1
  value is not a valid dict (type=type_error.dict)
type
  field required (type=value_error.missing)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    HumanMessage(content="주문할게요", type="human"), 
    AIMessage(content=product_list, type="ai"),
]

try:
    output = order_create_router_runnable.invoke({
        "input_message": HumanMessage(content=input_message, type="human"),
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

Exception: Error code: 400 - {'error': {'message': "Missing required parameter: 'messages[2].content[0].type'.", 'type': 'invalid_request_error', 'param': 'messages[2].content[0].type', 'code': 'missing_required_parameter'}}


messages 리스트에 하나만 메시지 담기니 괜찮네?

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage


input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    HumanMessage(content="주문할게요"), 
    ]


try:
    output = order_create_router_runnable.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

content='' additional_kwargs={'tool_calls': [{'id': 'call_DA7MVc1jDNuDIjoOSv8UBDlx', 'function': {'arguments': '{"task_to_request":"제품 목록을 제시해주세요.","evaluation_for_choice":"good","rationale_for_evaluation":"고객이 주문을 원했고, 상품 목록을 제시하여 주문을 진행할 수 있도록 도와야 합니다."}', 'name': 'ToPresentProductList'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 374, 'total_tokens': 459}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-bad8eefe-7753-4e94-b47a-0fc029659004-0' tool_calls=[{'name': 'ToPresentProductList', 'args': {'task_to_request': '제품 목록을 제시해주세요.', 'evaluation_for_choice': 'good', 'rationale_for_evaluation': '고객이 주문을 원했고, 상품 목록을 제시하여 주문을 진행할 수 있도록 도와야 합니다.'}, 'id': 'call_DA7MVc1jDNuDIjoOSv8UBDlx'}]


이건 또 바로 AIMessage를 뱉네? 위에는 튜플에 감싸져 있었는데

In [ ]:
output[0].tool_calls

TypeError: 'AIMessage' object is not subscriptable

In [ ]:
output.tool_calls

[{'name': 'ToPresentProductList',
  'args': {'task_to_request': '제품 목록을 제시해주세요.',
   'evaluation_for_choice': 'good',
   'rationale_for_evaluation': '고객이 주문을 원했고, 상품 목록을 제시하여 주문을 진행할 수 있도록 도와야 합니다.'},
  'id': 'call_DA7MVc1jDNuDIjoOSv8UBDlx'}]

product_list 때문이었음

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage


input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    AIMessage(content="안녕하세요"),
    ]


try:
    output = order_create_router_runnable.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

content='' additional_kwargs={'tool_calls': [{'id': 'call_17jXoVATLj1HzuysCWqbp3Ss', 'function': {'arguments': '{"task_to_request":"판매 중인 상품 목록을 제시해주세요.","evaluation_for_choice":"good","rationale_for_evaluation":"고객이 주문할 상품을 선택하기 위해 상품 목록이 필요합니다."}', 'name': 'ToPresentProductList'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 374, 'total_tokens': 449}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-fcb1b9d8-0adb-440b-8502-493629ddd684-0' tool_calls=[{'name': 'ToPresentProductList', 'args': {'task_to_request': '판매 중인 상품 목록을 제시해주세요.', 'evaluation_for_choice': 'good', 'rationale_for_evaluation': '고객이 주문할 상품을 선택하기 위해 상품 목록이 필요합니다.'}, 'id': 'call_17jXoVATLj1HzuysCWqbp3Ss'}]


In [ ]:
output.tool_calls

[{'name': 'ToPresentProductList',
  'args': {'task_to_request': '판매 중인 상품 목록을 제시해주세요.',
   'evaluation_for_choice': 'good',
   'rationale_for_evaluation': '고객이 주문할 상품을 선택하기 위해 상품 목록이 필요합니다.'},
  'id': 'call_17jXoVATLj1HzuysCWqbp3Ss'}]

In [ ]:
product_list

[{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000},
 {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500},
 {'product_name': '미니 백설기', 'quantity': 35, 'price': 31500},
 {'product_name': '개별 모듬팩', 'quantity': 1, 'price': 13500}]

In [ ]:
product_list

{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000}

아 문자열이 아닌 가령 리스트나 딕셔너리 형태라 그런 듯

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage


input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    AIMessage(content=product_list[0]),
    ]


try:
    output = order_create_router_runnable.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

ValidationError: 2 validation errors for AIMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)

In [ ]:
import json

product_list = [
    {
        "product_name": "떡케익5호",
        "quantity": 1,
        "price": 54000
    },
    {
        "product_name": "무지개 백설기 케익",
        "quantity": 1,
        "price": 51500
    },
    {
        "product_name": "미니 백설기",
        "quantity": 35,
        "price": 31500
    },
    {
        "product_name": "개별 모듬팩",
        "quantity": 1,
        "price": 13500
    }
]

product_list_str = json.dumps(product_list, ensure_ascii=False)
product_list_str


'[{"product_name": "떡케익5호", "quantity": 1, "price": 54000}, {"product_name": "무지개 백설기 케익", "quantity": 1, "price": 51500}, {"product_name": "미니 백설기", "quantity": 35, "price": 31500}, {"product_name": "개별 모듬팩", "quantity": 1, "price": 13500}]'

In [59]:
from langchain_core.messages import HumanMessage, AIMessage


input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    AIMessage(content=product_list_str),
    ]


try:
    output = order_create_router_runnable.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

content='' additional_kwargs={'tool_calls': [{'id': 'call_Cstpg3dqfOCMCYCHk42GKaez', 'function': {'arguments': '{"task_to_request":"떡케익 2개 주문할게요","evaluation_for_choice":"good","rationale_for_evaluation":"고객이 주문할 상품을 명시했기 때문에 상품 목록을 제시하는 것이 적절합니다."}', 'name': 'ToPresentProductList'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 506, 'total_tokens': 592}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-6748799a-d00c-4770-8391-f2cfb022b717-0' tool_calls=[{'name': 'ToPresentProductList', 'args': {'task_to_request': '떡케익 2개 주문할게요', 'evaluation_for_choice': 'good', 'rationale_for_evaluation': '고객이 주문할 상품을 명시했기 때문에 상품 목록을 제시하는 것이 적절합니다.'}, 'id': 'call_Cstpg3dqfOCMCYCHk42GKaez'}]


In [60]:
 output.tool_calls

[{'name': 'ToPresentProductList',
  'args': {'task_to_request': '떡케익 2개 주문할게요',
   'evaluation_for_choice': 'good',
   'rationale_for_evaluation': '고객이 주문할 상품을 명시했기 때문에 상품 목록을 제시하는 것이 적절합니다.'},
  'id': 'call_Cstpg3dqfOCMCYCHk42GKaez'}]

In [61]:
from langchain_core.messages import HumanMessage, AIMessage


input_message = "떡케익 2개 주문할게요"
user_info = "1"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    ]


try:
    output = order_create_router_runnable.invoke({
        "input_message": input_message,
        "user_info": user_info,
        "messages": messages,
    })
    print(output)
except Exception as e: 
    print(f"Exception: {e}")

content='' additional_kwargs={'tool_calls': [{'id': 'call_5F8Yt24QQkQoQh10bYvuruao', 'function': {'arguments': '{"task_to_request":"제시된 상품 목록 중 떡케익 5호 1개를 주문합니다.","evaluation_for_choice":"good","rationale_for_evaluation":"고객이 주문하고자 하는 상품을 명확히 선택하고 주문을 진행하려는 의사를 밝힘."}', 'name': 'ToPresentProductList'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 514, 'total_tokens': 619}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-db6a509a-cebb-4487-9520-f22dc220ea7a-0' tool_calls=[{'name': 'ToPresentProductList', 'args': {'task_to_request': '제시된 상품 목록 중 떡케익 5호 1개를 주문합니다.', 'evaluation_for_choice': 'good', 'rationale_for_evaluation': '고객이 주문하고자 하는 상품을 명확히 선택하고 주문을 진행하려는 의사를 밝힘.'}, 'id': 'call_5F8Yt24QQkQoQh10bYvuruao'}]


In [62]:
 output.tool_calls

[{'name': 'ToPresentProductList',
  'args': {'task_to_request': '제시된 상품 목록 중 떡케익 5호 1개를 주문합니다.',
   'evaluation_for_choice': 'good',
   'rationale_for_evaluation': '고객이 주문하고자 하는 상품을 명확히 선택하고 주문을 진행하려는 의사를 밝힘.'},
  'id': 'call_5F8Yt24QQkQoQh10bYvuruao'}]

---

In [63]:
from langchain_core.pydantic_v1 import BaseModel, Field

class EvalueateCalssification(BaseModel):
    """Records misclassification of router"""
    input: str = Field(description="라우터에 입력된 메시지")
    classification_of_router: str = Field(description="라우터의 분류 결과")
    evaluation_for_classification: str = Field(description=" 라우터의 분류에 대한 평가. '올바름'과 '잘못됨' 중 하나로 평가")
    reason_for_evaluation: str = Field(description="라우터 분류에 대한 평가의 이유")

In [64]:
from langchain_core.prompts import ChatPromptTemplate

check_routing_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 라우팅 결과가 올바른지 꼼꼼하게 점검해야 해.
            라우팅 기준: {routing_criteria}
            입력 메시지: {input_message}
            모델의 분류 결과: {model_classification}

            도구를 사용해 모델의 분류 결과를 평가해.      
            
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
).partial(routing_criteria=routing_criteria)

# model="gpt-4o"
# gpt4o = ChatOpenAI(model=model)
# check_routing_runnable = check_routing_prompt | gpt4o.bind_tools([EvalueateCalssification])
check_routing_runnable = check_routing_prompt | llm.bind_tools([EvalueateCalssification])

routing_criteria = """
    이전 대화에서 판매 중인 상품 목록을 제시했는지 확인해.
    만약 판매중인 상품 목록을 제시하지 않았다면 ToPresentProductList를 사용해
    고객이 주문할 품목을 말했다면 TodOrderRequestConfirmation를 사용해.
    고객이 주문을 진행하려는 내용에 동의했다면 create_order를 사용해.
    """

In [65]:
input_message = "떡케익 2개 주문할게요"
model_classification = "ToPresentProductList"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.tool_calls

[{'name': 'EvalueateCalssification',
  'args': {'input': '떡케익 2개 주문할게요',
   'classification_of_router': 'ToPresentProductList',
   'evaluation_for_classification': '잘못됨',
   'reason_for_evaluation': '고객이 주문을 진행하려는 내용에 동의했기 때문에 TodOrderRequestConfirmation을 사용해야 합니다.'},
  'id': 'call_PgJphnXkcp7Tx03cGgYpDahq'}]

틀림.

In [68]:
input_message = "응"
model_classification = "TodOrderRequestConfirmation"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    HumanMessage(content="떡케익 2개 주문할게요"),
    AIMessage(content="떡케익 2개로 주문하면 될까요?")
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.tool_calls

[{'name': 'EvalueateCalssification',
  'args': {'input': '떡케익 2개 주문할게요',
   'classification_of_router': 'TodOrderRequestConfirmation',
   'evaluation_for_classification': '올바름',
   'reason_for_evaluation': '주문 요청 확인'},
  'id': 'call_JssY2VIu14SRhyddzr3fON5i'}]

In [67]:
input_message = "응"
model_classification = "create_order"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    HumanMessage(content="떡케익 2개 주문할게요"),
    AIMessage(content="떡케익 2개로 주문하면 될까요?")
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.tool_calls

[{'name': 'EvalueateCalssification',
  'args': {'input': '떡케익 2개 주문할게요',
   'classification_of_router': 'create_order',
   'evaluation_for_classification': '올바름',
   'reason_for_evaluation': '주문을 진행하는 내용에 동의함'},
  'id': 'call_NIIzvAW2wUgzzVGbKTTkbak4'}]

모델 gpt4o로 교체 후 다시 시도. 잘못 잡아냄. 뽀록인 듯?

In [70]:
from langchain_core.prompts import ChatPromptTemplate

check_routing_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 라우팅 결과가 올바른지 꼼꼼하게 점검해야 해.
            라우팅 기준: {routing_criteria}
            입력 메시지: {input_message}
            모델의 분류 결과: {model_classification}

            도구를 사용해 모델의 분류 결과를 평가해.      
            
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
).partial(routing_criteria=routing_criteria)

model="gpt-4o"
gpt4o = ChatOpenAI(model=model)
check_routing_runnable = check_routing_prompt | gpt4o.bind_tools([EvalueateCalssification])
# check_routing_runnable = check_routing_prompt | llm.bind_tools([EvalueateCalssification])

In [71]:
input_message = "응"
model_classification = "TodOrderRequestConfirmation"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    HumanMessage(content="떡케익 2개 주문할게요"),
    AIMessage(content="떡케익 2개로 주문하면 될까요?")
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.tool_calls

[{'name': 'EvalueateCalssification',
  'args': {'input': '응',
   'classification_of_router': 'TodOrderRequestConfirmation',
   'evaluation_for_classification': '잘못됨',
   'reason_for_evaluation': "고객이 주문을 진행하려는 내용을 확인했으므로 'create_order'로 라우팅해야 합니다."},
  'id': 'call_DgmDUEeIaIB6DIiaU3yWBaid'}]

쉬운 데도 gpt4o도 틀리는 경우 있네

In [73]:
input_message = "응"
model_classification = "create_order"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    HumanMessage(content="떡케익 2개 주문할게요"),
    AIMessage(content="떡케익 2개로 주문하면 될까요?")
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.tool_calls

[{'name': 'EvalueateCalssification',
  'args': {'input': '응',
   'classification_of_router': 'create_order',
   'evaluation_for_classification': '잘못됨',
   'reason_for_evaluation': '고객이 주문을 진행하려는 내용에 동의했다는 명확한 표현이 없었기에 create_order 대신 ToOrderRequestConfirmation이 사용되어야 합니다.'},
  'id': 'call_emQYFWHaFwz6Vcmz5bMRq2uP'}]

In [74]:
input_message = "응"
model_classification = "create_order"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    HumanMessage(content="떡케익 2개 주문할게요"),
    AIMessage(content="떡케익 2개로 주문하면 될까요?")
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.tool_calls

[{'name': 'EvalueateCalssification',
  'args': {'input': '응',
   'classification_of_router': 'create_order',
   'evaluation_for_classification': '잘못됨',
   'reason_for_evaluation': "고객이 주문할 품목을 명확히 언급했으므로, 'create_order'가 아닌 'TodOrderRequestConfirmation'으로 라우팅해야 합니다."},
  'id': 'call_0u7LjwtbMQa4B4kQTb0mPTkp'}]

In [80]:
from langchain_core.prompts import ChatPromptTemplate

check_routing_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 라우팅 결과가 올바른지 검사하는 전문가야.
            라우팅 기준: {routing_criteria}
            입력 메시지: {input_message}
            모델의 분류 결과: {model_classification}

            모델의 분류 결과가 맞다면 good, 틀리다면 bad를 출력해. 
            반드시 응답은 good과 bad 중 하나여야 해.
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
).partial(routing_criteria=routing_criteria)

# model="gpt-4o"
# gpt4o = ChatOpenAI(model=model)
check_routing_runnable = check_routing_prompt | llm

In [81]:
input_message = "응"
model_classification = "create_order"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    HumanMessage(content="떡케익 2개 주문할게요"),
    AIMessage(content="떡케익 2개로 주문하면 될까요?")
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.content

'create_order'

In [82]:
output

AIMessage(content='create_order', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 415, 'total_tokens': 417}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c2a99493-a2d0-4ecd-ba9b-a4654a46d1b9-0')

In [83]:
from langchain_core.prompts import ChatPromptTemplate

check_routing_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 라우팅 결과가 올바른지 검사하는 전문가야.
            라우팅 기준: {routing_criteria}
            입력 메시지: {input_message}
            모델의 분류 결과: {model_classification}

            모델의 분류 결과가 맞다면 good, 틀리다면 bad를 출력해. 
            반드시 응답은 good과 bad 중 하나여야 해.
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
).partial(routing_criteria=routing_criteria)

model="gpt-4o"
gpt4o = ChatOpenAI(model=model)
check_routing_runnable = check_routing_prompt | gpt4o

In [84]:
input_message = "응"
model_classification = "create_order"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    HumanMessage(content="떡케익 2개 주문할게요"),
    AIMessage(content="떡케익 2개로 주문하면 될까요?")
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.content

'TodOrderRequestConfirmation'

In [85]:
from langchain_core.prompts import ChatPromptTemplate

check_routing_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 라우팅 결과가 올바른지 검사하는 전문가야.
            라우팅 기준: {routing_criteria}
            입력 메시지: {input_message}
            라우팅 결과: {model_classification}

            라우팅 결과가 맞다면 good, 틀리다면 bad를 출력해. 
            반드시 응답은 good과 bad 중 하나여야 해.
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
).partial(routing_criteria=routing_criteria)

model="gpt-4o"
gpt4o = ChatOpenAI(model=model)
check_routing_runnable = check_routing_prompt | gpt4o

In [86]:
input_message = "응"
model_classification = "create_order"
messages = [
    AIMessage(content="상품 목록\n"+product_list_str),
    HumanMessage(content="떡케익 2개 주문할게요"),
    AIMessage(content="떡케익 2개로 주문하면 될까요?")
    ]

output = check_routing_runnable.invoke({"input_message": input_message,
                                        "model_classification": model_classification,
                                        "messages": messages})
output.content

'TodOrderRequestConfirmation'

---

# 담당 작업 외 예외 처리

In [87]:
from chain.langgraph_states import State
from chain.langgraph_assistants import request_order_confirmation_runnable

def request_order_confirmation(state: State):
    print("-"*77)
    print("request_order_confirmation 진입")
    print("state\n", state)

    messages = state["messages"]
    response = request_order_confirmation_runnable.invoke({"messages": messages})
    
    return {"messages": response}

ModuleNotFoundError: No module named 'chain'

In [88]:
class CompleteOrEscalate(BaseModel):
    """A tool to mark the current task as completed and/or to escalate control of the dialog to the main assistant,
    who can re-route the dialog based on the user's needs."""

    cancel: bool = True
    reason: str

    class Config:
        schema_extra = {
            "example": {
                "cancel": True,
                "reason": "User changed their mind about the current task.",
            },
            "example 2": {
                "cancel": True,
                "reason": "User made a statement that is irrelevant to the current task.",
            },
            "example 3": {
                "cancel": False,
                "reason": "I need to use the tool to place a new order.",
            },
        }


In [89]:
request_order_confirmation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
        """
        너는 사용자의 요청 사항을 확인하는 주문봇이야.
        ToolMessage에 담긴 정보를 바탕으로 아례 예시처럼 응답을 생성해줘.
        응답에는 반드시 상품명, 가격, 수량을 모두 포함해야 해.
        응답을 출력하기 전에 응답에 상품명, 가격, 수량이 모두 포함돼 있는지 확인하고, 깜빡했다면 반드시 포함시켜줘.

        # ToolMessage에 담긴 정보 예시
        product_list: 떡케익5호 - 54,000원, 무지개 백설기 케익 - 51,500원, 미니 백설기 (35개 세트) - 31,500원, 개별 모듬팩 - 13,500원
        customer_order_request: 미니 백설기 1개, 개별 모듬팩 2개
        message_to_be_confirmed: 미니 백설기 1개 - 31,500원, 개별 모듬팩 2개 - 27,000원

        # 모델 응답 예시

        주문 품목:
        1. 상품명: 미니 백설기 
           가격: 31,500원 
           수량: 1개

        2. 상품명: 개별 모듬팩
           가격: 13,500원
           수량: 2개

           총 가격: 31,500+27,000 = 58,500원
        
        이대로 주문을 진행하면 될까요?
        """
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
    
)
request_order_confirmation_runnable = request_order_confirmation_prompt | llm.bind_tools([CompleteOrEscalate])


In [90]:
product_list_str

'[{"product_name": "떡케익5호", "quantity": 1, "price": 54000}, {"product_name": "무지개 백설기 케익", "quantity": 1, "price": 51500}, {"product_name": "미니 백설기", "quantity": 35, "price": 31500}, {"product_name": "개별 모듬팩", "quantity": 1, "price": 13500}]'

In [91]:
messages = [
    HumanMessage(content="주문할게요"),
    AIMessage(content=product_list_str),
    HumanMessage(content="떡케익 2개 할게요"),
]
output = request_order_confirmation_runnable.invoke({"messages": messages})
output

AIMessage(content='주문 품목:\n1. 상품명: 떡케익5호\n   가격: 54,000원\n   수량: 2개\n\n총 가격: 54,000 x 2 = 108,000원\n\n이대로 주문을 진행하면 될까요?', response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 694, 'total_tokens': 780}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-18219ad1-1867-4619-82ca-d40b668ad7de-0')

In [93]:
messages = [
    HumanMessage(content="주문할게요"),
    AIMessage(content=product_list_str),
    HumanMessage(content="아 그냥 주문 안 할래"),
]
output = request_order_confirmation_runnable.invoke({"messages": messages})
output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gtbZIB6cpAUHSSwiOwRKCuGN', 'function': {'arguments': '{"cancel":true,"reason":"사용자가 주문을 취소했습니다."}', 'name': 'CompleteOrEscalate'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 690, 'total_tokens': 724}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8fcf83bf-8345-4d3c-877d-928240cdf905-0', tool_calls=[{'name': 'CompleteOrEscalate', 'args': {'cancel': True, 'reason': '사용자가 주문을 취소했습니다.'}, 'id': 'call_gtbZIB6cpAUHSSwiOwRKCuGN'}])

In [94]:
messages = [
    HumanMessage(content="주문할게요"),
    AIMessage(content=product_list_str),
    HumanMessage(content="판매 중인 상품이 뭐가 있더라?"),
]
output = request_order_confirmation_runnable.invoke({"messages": messages})
output

AIMessage(content='주문할 상품 목록은 다음과 같아:\n\n1. 상품명: 떡케익5호\n   가격: 54,000원\n   수량: 1개\n\n2. 상품명: 무지개 백설기 케익\n   가격: 51,500원\n   수량: 1개\n\n3. 상품명: 미니 백설기\n   가격: 31,500원\n   수량: 35개\n\n4. 상품명: 개별 모듬팩\n   가격: 13,500원\n   수량: 1개\n\n주문을 원하시는 상품을 선택해주세요.', response_metadata={'token_usage': {'completion_tokens': 192, 'prompt_tokens': 699, 'total_tokens': 891}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-27ae4ef6-26ba-4625-acf9-85e9ff0ec030-0')